In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import cv2
import resnet_V2
import data_util

In [2]:
tf.random.set_seed(42)

In [3]:
mse = keras.losses.MeanSquaredError()
ce = keras.losses.CategoricalCrossentropy()

In [16]:
def custom_loss(y_true, y_pred):
    y_class_true = y_true[:, 0:133]
    y_class_pred = y_pred[:, 0:133]
    
    y_point_true = y_true[:, 133:]
    y_point_pred = y_pred[:, 133:]
    
    class_loss = ce(y_class_true, y_class_pred)
    ce_loss = mse(y_point_true, y_point_pred)
    
    return 0.5 * ce_loss + 0.5 * class_loss

In [18]:
input_size= 256
batch_size=16

ae = resnet_V2.ResNetAE()
encoder = ae.encoder
input = keras.layers.Input(shape=(256,256,3))

encoder_output = encoder(input)
flatten = keras.layers.Flatten()(encoder_output)

branch1_class = keras.layers.Dense(133,activation='softmax',name='branch1_class')(flatten)

branch2_landmark = keras.layers.Dense(16,name='branch2_landmark')(flatten)

out= keras.layers.concatenate([branch1_class,branch2_landmark])       # concatnate?

model = keras.models.Model(input, out)
model.compile(loss=custom_loss,
              optimizer='adam'
              )
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
res_net_encoder_1 (ResNetEncode (None, 16, 16, 128)  1479600     input_2[0][0]                    
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 32768)        0           res_net_encoder_1[0][0]          
__________________________________________________________________________________________________
branch1_class (Dense)           (None, 133)          4358277     flatten_1[0][0]                  
_______________________________________________________________________________________

In [9]:
ds = data_util.get_cu_dataset(train_type='class', batch_size=batch_size)

In [10]:
for i in ds.take(1):
    print(i[0].shape)
    print(i[1].shape)

(16, 256, 256, 3)
(16, 149)


In [11]:
ds_size = len(list(ds))
ds_size

478

In [12]:
train_size = int(0.7 * ds_size)
val_size = int(0.15 * ds_size)
test_size = int(0.15 * ds_size)

train_ds = ds.take(train_size)
test_ds = ds.skip(train_size)
val_ds = test_ds.skip(val_size)
test_ds = test_ds.take(test_size)

In [13]:
len(list(train_ds)), len(list(val_ds)), len(list(test_ds))

(334, 73, 71)

In [14]:
ckpt_path = './ckpt/multi'
callbacks_list = [
    # accuracy 기준 가장 높은 모델의 weight 저장
    tf.keras.callbacks.ModelCheckpoint(
        filepath = ckpt_path,
        monitor='val_loss',
        mode='min',
        save_weights_only=True,
        save_best_only=True
    ),
    # EarlyStopping
    # tf.keras.callbacks.EarlyStopping(
    #    monitor='val_loss',
    #    mode='min',
    #    verbose=1, 
    #    patience=20
    # )
]

In [19]:
hist = model.fit(train_ds, validation_data=val_ds, epochs=100, callbacks=callbacks_list)

Epoch 1/100
      1/Unknown - 0s 0s/step - loss: 6057.9741

KeyboardInterrupt: 